### 데이터 확인 및 불러오기

- 데이터는 구글 드라이브 > 프로젝트 > 중급 프로젝트 > 원본데이터에 있어요.
- 제공된 문서 파일들과 `data_list.csv` 파일에 담긴 메타데이터를 확인합니다.
- 문서 파일을 불러옵니다. `hwp`와 `pdf` 두 가지 포맷에 대응할 수 있어야 해요.
- 유용한 메타데이터가 무엇일지 판단하여 함께 활용해 보세요.

### 문서 청킹

- 청크 크기, 그리고 청크 간 중첩 크기를 잘 설정하여 문서를 청킹합니다.
- (심화) 제안서 포맷을 활용해 의미 단위까지 고려하여 문서를 청킹해 보고 성능을 확인해 보세요.

### 임베딩 생성

- 임베딩 성능과 비용을 고려해 적절한 임베딩 모델을 선정합니다.
- 사용의 편의성과 검색 기능을 고려해 적절한 Vector DB를 선정합니다.

### Retrieval 기능 구현

- 먼저 naive한 retrieval을 구현해 베이스라인으로 삼으면 좋습니다. 베이스라인을 기반으로 점점 더 retrieval 기능을 고도화해 나가면 됩니다.
- 데이터 소스 문서가 하나가 아니라 여러 개이기 때문에, 타깃 문서를 정확히 찾기 위해 메타데이터 필터링을 활용할 수 있습니다. 이때 사용자가 발주 기관이나 사업명 같은 정보를 비슷하지만 정확하지는 않게 입력하는 케이스도 고려하는 게 좋아요.
- (심화) Retrieval과 관련해 다양한 옵션과 기법을 실험하면서 성능을 확인해 보세요.
    - Retrieval을 위한 프롬프트 엔지니어링
    - Top-k 검색에서 k 값 설정
    - 단순 유사도 기반 검색 / MMR(Maximum Marginal Relevance) / Hybrid Search
    - Multi-Query, Re-Ranking 등 심화 기법

### Generation 기능 구현

- 텍스트 생성 능력과 비용을 고려해 적절한 언어 모델을 선정합니다.
- 원하는 답변 양상과 분량에 따라서 temperature, top_p, max tokens 등 텍스트 생성과 관련된 옵션도 적절하게 설정해 주세요.
- 답변 생성을 위한 최적의 프롬프트를 작성합니다.
    - Retrieval 과정에서 찾은 컨텍스트를 충실히 반영해야 합니다.
    - 불필요한 내용이 답변에 포함되지 않도록 합니다.
    - 응답의 톤이나 스타일을 조정합니다.
    - 비용을 고려해 토큰 사용량도 최적화해 보세요.
- RAG 시스템이 대화 맥락을 유지하여 사용자와 대화를 이어 갈 수 있도록 해 주세요.

### 성능 평가

- 답변의 품질을 평가할 때는 다음과 같은 기준을 사용할 수 있습니다.
    - 사용자가 요청한 내용을 단일 문서에서 정확하게 뽑아내 답변하는지
    - 사용자가 여러 문서에 대해 요청한 내용을 잘 종합해서 답변하는지
    - 후속 질문의 맥락을 잘 이해하고 답변하는지
    - 문서에 포함되어 있지 않은 내용에 대해서는 모른다고 답변하는지
- 위와 같은 기준을 어떤 방식으로 평가하고 어떤 지표로 나타낼지 선정해 보세요.
- 주어진 문서 데이터에 맞게 다양한 질문 세트를 준비하여 성능 평가에 활용해 보세요.
    - 질문 예시
        - 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.
            - 콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.
            - 교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?
        - 기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?
            - 그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘.
        - 한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘.
        - 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
            - 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘.
- 품질 좋은 답변도 중요하지만 응답 속도가 너무 느려져서도 안 됩니다.

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

data_dir = '/content/drive/MyDrive/Data'
print(f"Listing contents of directory: {data_dir}")
try:
    for item in os.listdir(data_dir):
        print(item)
except FileNotFoundError:
    print(f"Directory not found: {data_dir}")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Listing contents of directory: /content/drive/MyDrive/Data
files
data_list.csv
data_list.gsheet


# Install Packages

In [9]:
%pip install easyocr

In [10]:
%pip install pyhwp hwp-extract

In [11]:
%pip install pymupdf

## Load data_list.csv and find NaN values


In [8]:
import pandas as pd
import os

data_dir = '/content/drive/MyDrive/Data'
csv_path = os.path.join(data_dir, 'data_list.csv')

try:
    # Load the CSV file into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)

    # Display the first few rows of the DataFrame
    print("First 5 rows of data_list.csv:")
    display(data_list_df.head())

    # Check for NaN values in each column
    print("\nChecking for NaN values:")
    nan_counts = data_list_df.isnull().sum()

    # Display the count of NaN values per column
    print("Number of NaN values per column:")
    print(nan_counts)

except FileNotFoundError:
    print(f"Error: The file {csv_path} was not found.")
except Exception as e:
    print(f"An error occurred while reading the CSV file or checking for NaN values: {e}")

First 5 rows of data_list.csv:


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...



Checking for NaN values:
Number of NaN values per column:
공고 번호        18
공고 차수        18
사업명           0
사업 금액         1
발주 기관         0
공개 일자         0
입찰 참여 시작일    26
입찰 참여 마감일     8
사업 요약         0
파일형식          0
파일명           0
텍스트           0
dtype: int64


# Load Document and Chunking

In [12]:
import os
import sys
import fitz  # PyMuPDF
import easyocr
import numpy as np
from PIL import Image
import subprocess

def find_executable(executable_name):
    """Finds the full path to an executable within the environment's PATH or script directories."""
    # Check in PATH first
    for path in os.environ["PATH"].split(os.pathsep):
        exe_path = os.path.join(path, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    # Check in common script directories relative to sys.executable
    script_dirs = [
        os.path.join(os.path.dirname(sys.executable), 'bin'), # Linux/macOS
        os.path.join(os.path.dirname(sys.executable), 'Scripts') # Windows
    ]
    for script_dir in script_dirs:
        exe_path = os.path.join(script_dir, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    return None


# Define HWP extraction function using pyhwp based on typical usage
def extract_text_from_hwp(hwp_path):
    """
    Extracts text from an HWP file using pyhwp's hwp5txt command.

    Args:
        hwp_path (str): The path to the HWP file.

    Returns:
        str: The extracted text from the HWP.
    """
    text = ""
    hwp5txt_path = find_executable('hwp5txt')

    if not hwp5txt_path:
        print("Error: hwp5txt command not found. Make sure pyhwp is installed correctly.")
        return ""

    try:
        # Use hwp5txt command-line tool via subprocess
        result = subprocess.run([hwp5txt_path, hwp_path], capture_output=True, text=True, encoding='utf-8')
        if result.returncode == 0:
            text = result.stdout
        else:
            print(f"Error extracting text from HWP {hwp_path} using hwp5txt: {result.stderr}")
            text = ""
    except Exception as e:
        print(f"Error processing HWP {hwp_path}: {e}")
        text = ""
    return text

# Define function for PDF text extraction using easyOCR
def extract_text_from_pdf_easyocr(pdf_path):
    """
    Extracts text from a PDF file using easyOCR.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text from the PDF.
    """
    # Initialize easyocr reader for Korean and English (adjust languages as needed)
    reader = easyocr.Reader(['ko', 'en'])
    text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)

            # Render page to an image (as a numpy array) for easyOCR
            pixmap = page.get_pixmap()
            img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
            img_np = np.array(img)

            # Use easyOCR to read text from the image
            result = reader.readtext(img_np, detail=0) # detail=0 returns only the text
            text += " ".join(result) + "\n" # Join lines with a space and add newline between pages

        pdf_document.close()
    except Exception as e:
        print(f"Error processing PDF {pdf_path} with easyOCR: {e}")
        text = "" # Return empty string in case of processing error

    return text


data_dir = '/content/drive/MyDrive/Data/files'
extracted_texts = {}

print(f"Attempting to process files in directory: {data_dir}")

if not os.path.isdir(data_dir):
    print(f"Error: Directory not found at {data_dir}")
else:
    files_in_dir = os.listdir(data_dir)
    if not files_in_dir:
        print(f"No files found in directory: {data_dir}")
    else:
        for filename in files_in_dir:
            file_path = os.path.join(data_dir, filename)
            if os.path.isfile(file_path):
                if filename.lower().endswith('.pdf'):
                    try:
                        text = extract_text_from_pdf_easyocr(file_path) # Use easyOCR function
                        extracted_texts[filename] = text
                    except Exception as e:
                        print(f"Failed to process PDF {filename} with easyOCR: {e}")
                        extracted_texts[filename] = "" # Store empty string for failed files
                elif filename.lower().endswith('.hwp'):
                    try:
                        text = extract_text_from_hwp(file_path) # Use pyhwp function
                        extracted_texts[filename] = text
                    except Exception as e:
                        print(f"Failed to process HWP {filename} with pyhwp: {e}")
                        extracted_texts[filename] = "" # Store empty string for failed files
                else:
                    print(f"Skipping unsupported file type: {filename}")


        print("\nExtraction complete.")
        print(f"Total files found in directory: {len(files_in_dir)}")
        print(f"Files processed (PDF/HWP): {len(extracted_texts)}")

        # Display the keys (filenames) of the extracted texts dictionary
        print("\nFilenames with extracted text:")
        for fname in extracted_texts.keys():
            print(fname)

Attempting to process files in directory: /content/drive/MyDrive/Data/files
Error extracting text from HWP /content/drive/MyDrive/Data/files/한국농어촌공사_아세안+3 식량안보정보시스템(AFSIS) 3단계 협력(캄보디아.hwp using hwp5txt: Traceback (most recent call last):
  File "/usr/local/bin/hwp5txt", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/hwp5/hwp5txt.py", line 84, in main
    transform(hwp5file, dest)
  File "/usr/local/lib/python3.12/dist-packages/hwp5/transforms/__init__.py", line 50, in transform_hwp5
    return transform_xhwp5(xhwp5path, output)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/hwp5/plat/_lxml.py", line 95, in transform_into_stream
    return self._transform(inp_file, output)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/hwp5/plat/_lxml.py", line 101, in _transform
    source = etree.parse(input

In [22]:
# TODO: Implement embedding using text-embedding-3-small
# This will involve:
# 1. Initializing the embedding model.
# 2. Iterating through the extracted text chunks.
# 3. Generating embeddings for each chunk.
# 4. Storing the embeddings and associated metadata (like filename and chunk index) in a suitable structure.

# Make sure to install the openai library: %pip install openai
import os
from openai import OpenAI
from google.colab import userdata
import re # Import re for splitting text

# Access your API key from Colab secrets
# You'll need to add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors

# Initialize the OpenAI client
if OPENAI_API_KEY:
    client = OpenAI(api_key=OPENAI_API_KEY)

    # Initialize the embedding model
    embedding_model_name = "text-embedding-3-small"

    # Create a structure to store embeddings and metadata
    embedded_chunks = []

    # Define a simple chunking function
    def chunk_text(text, max_tokens=8000, overlap=100):
        """
        Simple text chunking function. Splits text by paragraphs and then
        further splits long paragraphs by character length if necessary.
        This is a basic implementation and can be improved.
        """
        chunks = []
        paragraphs = text.split('\n\n') # Split by paragraphs

        for para in paragraphs:
            if not para.strip(): # Skip empty paragraphs
                continue

            # Estimate token count (very rough estimate, character-based)
            # A more accurate approach would use a proper tokenizer
            if len(para) > max_tokens * 4: # If paragraph is still too long (rough char estimate for max tokens)
                # Split long paragraphs by a large character chunk
                words = para.split()
                current_chunk = []
                current_length = 0
                for word in words:
                    # Simple character count approximation
                    if current_length + len(word) + 1 > max_tokens * 4:
                        chunks.append(" ".join(current_chunk))
                        current_chunk = [word]
                        current_length = len(word)
                    else:
                        current_chunk.append(word)
                        current_length += len(word) + 1
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
            else:
                chunks.append(para)

        # Optional: Add overlap between chunks (more complex to implement accurately with this simple method)
        # For this basic implementation, we'll skip explicit overlap for now.
        # A more sophisticated chunking method (like recursive character text splitter) is recommended for production

        return chunks


    # Iterate through extracted texts and generate embeddings
    for filename, text in extracted_texts.items():
        if text:  # Only process if text extraction was successful
            # Chunk the text before embedding
            text_chunks = chunk_text(text)
            print(f"Processing {filename}: {len(text_chunks)} chunks generated.")

            for i, chunk in enumerate(text_chunks):
                try:
                    # OpenAI embedding call
                    response = client.embeddings.create(
                        input=[chunk],
                        model=embedding_model_name,
                        encoding_format="float"
                    )
                    embedding = response.data[0].embedding
                    embedded_chunks.append({
                        "filename": filename,
                        "chunk_index": i, # Add chunk index
                        "text": chunk,
                        "embedding": embedding
                    })
                except Exception as e:
                    print(f"Error generating embedding for chunk {i} in {filename}: {e}")
                    # Optionally store the text without embedding or handle as needed

    print(f"Generated embeddings for {len(embedded_chunks)} total chunks across documents.")
    # You can inspect the first few embedded chunks
    if embedded_chunks:
        print("\nFirst 3 embedded chunks example:")
        for i in range(min(3, len(embedded_chunks))):
            print(f"--- Chunk {i+1} ---")
            print(f"Filename: {embedded_chunks[i]['filename']}")
            print(f"Chunk Index: {embedded_chunks[i]['chunk_index']}")
            print(f"Text snippet: {embedded_chunks[i]['text'][:200]}...") # Print first 200 chars
            print(f"Embedding dimension: {len(embedded_chunks[i]['embedding'])}")
else:
    print("Skipping embedding generation due to missing API key.")

Processing 대한장애인체육회_2025년 전국장애인체육대회 전산 및 시스템, 홈페이지 .hwp: 224 chunks generated.
Processing 인천광역시_인천일자리플랫폼 정보시스템 구축 ISP 수립용역.hwp: 236 chunks generated.
Processing 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.hwp: 277 chunks generated.
Processing 인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp: 234 chunks generated.
Processing KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .hwp: 327 chunks generated.
Processing 한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.hwp: 387 chunks generated.
Processing 재단법인경기도일자리재단_2025년 통합접수시스템 운영.hwp: 317 chunks generated.
Processing 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp: 207 chunks generated.
Processing 국방과학연구소_대용량 자료전송시스템 고도화.hwp: 126 chunks 

In [23]:
# Install FAISS
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.7 MB/s eta 0:00:00


In [24]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
# Ensure all embeddings are the same dimension
embedding_dimension = len(embedded_chunks[0]['embedding'])
embeddings_np = np.array([chunk['embedding'] for chunk in embedded_chunks if len(chunk['embedding']) == embedding_dimension]).astype('float32')

# Create a FAISS index
# Using IndexFlatL2 for L2 distance (Euclidean distance) similarity search
index = faiss.IndexFlatL2(embedding_dimension)

# Add the embeddings to the index
index.add(embeddings_np)

print(f"FAISS index created with {index.ntotal} embeddings.")

# Store the mapping from index ID to the original chunk data (filename, chunk_index, text)
# This is necessary to retrieve the original text and metadata after a FAISS search
index_to_chunk_map = [
    {'filename': chunk['filename'], 'chunk_index': chunk['chunk_index'], 'text': chunk['text']}
    for chunk in embedded_chunks if len(chunk['embedding']) == embedding_dimension
]

print(f"Created a mapping for {len(index_to_chunk_map)} chunks.")

# You can optionally save the index and the map to disk for later use
# faiss.write_index(index, "document.index")
# import pickle
# with open("index_to_chunk_map.pkl", "wb") as f:
#     pickle.dump(index_to_chunk_map, f)

FAISS index created with 25108 embeddings.
Created a mapping for 25108 chunks.


In [25]:
# TODO: Implement generation using gpt-4.1-mini
# This will involve:
# 1. Initializing the gpt-4.1-mini model.
# 2. Setting up the RAG pipeline:
#    - Taking a user query.
#    - Using the retriever (which will be built later) to find relevant text chunks based on the query's embedding.
#    - Constructing a prompt for the gpt-4.1-mini model that includes the user query and the retrieved text chunks as context.
#    - Generating a response using the gpt-4.1-mini model based on the prompt.

from openai import OpenAI
from google.colab import userdata

# Access your API key from Colab secrets for the generation model
# You'll need to add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors

# Initialize the OpenAI client for generation
if OPENAI_API_KEY:
    client_gen = OpenAI(api_key=OPENAI_API_KEY)
    generation_model_name = "gpt-4o-mini" # Using gpt-4o-mini as requested

    # Define the retrieval function using FAISS index and index_to_chunk_map
    def retrieve_chunks(query, k=5):
        """
        Retrieves top-k relevant chunks from the FAISS index based on the query.

        Args:
            query (str): The user query.
            k (int): The number of top relevant chunks to retrieve.

        Returns:
            list: A list of dictionaries, each containing 'filename', 'chunk_index', and 'text'
                  for the retrieved chunks.
        """
        if not OPENAI_API_KEY:
            print("Cannot perform retrieval: OpenAI API key is not set.")
            return []

        try:
            # Generate embedding for the query
            response = client.embeddings.create(
                input=[query],
                model="text-embedding-3-small", # Use the same embedding model as for documents
                encoding_format="float"
            )
            query_embedding = np.array([response.data[0].embedding]).astype('float32')

            # Perform similarity search in FAISS
            distances, indices = index.search(query_embedding, k)

            # Retrieve the corresponding chunk data
            retrieved_chunks = [index_to_chunk_map[i] for i in indices[0]]

            return retrieved_chunks

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

    # Define the generation function using gpt-4.1-mini
    def generate_response(query, retrieved_context):
        """
        Generates a response based on the query and retrieved context using gpt-4.1-mini.

        Args:
            query (str): The user query.
            retrieved_context (list): A list of text chunks retrieved from the documents.

        Returns:
            str: The generated response.
        """
        if not OPENAI_API_KEY:
            print("Cannot generate response: OpenAI API key is not set.")
            return "API key not configured."

        # Combine the retrieved context into a single string
        context_text = "\n\n---\n\n".join(retrieved_context)

        try:
            # Construct the prompt for the language model
            messages = [
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided documents."},
                {"role": "user", "content": f"Based on the following documents, answer the question:\n\nDocuments:\n{context_text}\n\nQuestion: {query}"}
            ]

            # Generate the response
            response = client_gen.chat.completions.create(
                model=generation_model_name,
                messages=messages,
                max_tokens=1000 # Adjust max_tokens as needed
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"Error during generation: {e}")
            return "An error occurred while generating the response."

    print("Retrieval and generation functions defined.")

else:
    print("Skipping RAG functionality definition due to missing API key.")

Retrieval and generation functions defined.


In [26]:
# Example Usage of the RAG pipeline

query = "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘."

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 276) ---
국민연금공단 이사장...
--- Chunk 2 from 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.hwp (Index: 40) ---
 1. 공단의 시스템 현황 
  □ 국민연금 정보시스템 구성도 및 인프라(H/W, S/W 등) 현황은 입찰 참가 사업자의 요청이 있는 경우 공단에서 제안서 작성에 필요하다고 판단되면 보안 서약서를 받고 담당자 입회하에 열람 가능
     『행정기관 및 공공기관 정보시스템 구축․운영 지침』(행정안전부고시 제2023-27호  2023.4.13.) 제17조 참고
 2. 추진체계
  
<표>
 
 2. 역할
 
<표>...
--- Chunk 3 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 305) ---
국민연금공단 귀하
【서식#11】
근로자 권리보호 이행 서약서
당사는 국민연금공단에서 시행하는 “2024년 이러닝시스템 운영 용역” 계약업체로서 근로자(하도급업체포함)의 인권보호·고용안정·노동환경 등의 권리보호를 위해 다음 사항을 이행할 것을 서약합니다....
--- Chunk 4 from 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.hwp (Index: 88) ---
  ※ 입찰공고일 현재 근무(최소 6개월 이상)하고 있는 전문인력의 숫자이며, 증빙서류를 첨부하여야 함(국민연금, 건강보험, 고용보험 가입증명서 중 1부 및 근무확인서 첨부)...
--- Chunk 5 from 한국해양조사협회_2024년 항해용 간행물 ᄑ

In [29]:
# Answer the next query

query = "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?"

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp (Index: 38) ---
󰏅 시스템 운영 요구사항...
--- Chunk 2 from 재단법인스포츠윤리센터_스포츠윤리센터 LMS(학습지원시스템) 기능개선.hwp (Index: 44) ---
시스템 장비구성 요구사항...
--- Chunk 3 from 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.hwp (Index: 59) ---
나. 데이터 요구사항(Data Requirement)...
--- Chunk 4 from 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp (Index: 66) ---
 4. 현 예약발매시스템 업무 프로세스 및 정보기술 현황 분석
    * 업무 요구사항 분석서, 정보기술 요구사항 분석서 등 도출 ...
--- Chunk 5 from 국방과학연구소_대용량 자료전송시스템 고도화.hwp (Index: 31) ---
기술 및 기능 요구사항
▢ 시스템 및 시스템운영 요구사항...

Generated Response:
문서 목록에서 "AI 기반 예측"에 대한 구체적인 요구사항이 언급되어 있지는 않습니다. 제공된 문서 제목들은 시스템 운영 요구사항, 장비구성 요구사항, 데이터 요구사항, 업무 프로세스 및 정보기술 현황 분석, 기술 및 기능 요구사항 등으로 구성되어 있습니다. 이러한 제목들이 AI 기반 예측에 대한 세부 요구사항을 포함하고 있는지 여부는 문서의 내용에 따라 다르므로, 구체적인 내용이 필요합니다. AI 기반 예측에 대한 명시적인 언급이 없

In [28]:
# Answer the next query

query = "교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?"

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 한국재정정보원_e나라도움 업무시스템 웹 접근성 컨설팅.hwp (Index: 38) ---
    ※ 발주기관의 사업담당자에게 과업수행 과정상 수반되는 전문기술, 최신동향 및 관련 사항을 이전(교육 대상, 시기 및 횟수 등은 상호협의하에 조정 가능)...
--- Chunk 2 from 파주도시관광공사_종량제봉투 판매관리 전산시스템 개선사업.hwp (Index: 294) ---
 “000‘ 사업과 관련하여 ”정보보안수칙“에 대한 교육실시 결과를 제출합니다....
--- Chunk 3 from 한국재정정보원_e나라도움 업무시스템 웹 접근성 컨설팅.hwp (Index: 137) ---
  가. 국가기관, 지방자치단체, 기획재정부장관이 지정한 공공기관 또는 지방공기업법령 적용 기관이 발급한 수행실적증명서...
--- Chunk 4 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 205) ---
※ 산출내역서 작성 시“제안 안내(p20~26)”를 반드시 참고하여 작성
※ 부서별 필수교육 콘텐츠 운영단가는 부서별로 확정된 인원에 대한 공단 콘텐츠 운영이므로 직무교육 운영단가에 준함
※ 외부콘텐츠․전문자격증 교육교재, 북러닝 및 전화외국어교육 등 실물 교재가 지급되는 항목은 반드시 실물 교재 제공에 배송료 포함 가격 제시
[별지 6
<표>
]...
--- Chunk 5 from 한국산업단지공단_산단 안전정보시스템 1차 구축 용역.hwp (Index: 330) ---
2. 보안교육 참석자 명단(용역 수행업체)...

Generated Response:
제공된 문서들

In [27]:
# Answer the next query

query = "콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘."

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 89) ---
 
 2-3. 콘텐츠 개발ㆍ관리...
--- Chunk 2 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 21) ---
 □ 직무 콘텐츠 개발‧운영 및 위탁교육(자기개발) 콘텐츠 운영...
--- Chunk 3 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 70) ---
  ○ 직무콘텐츠 60차시 및 부서별 필수 콘텐츠 10차시 개발(동영상 형태 포함)...
--- Chunk 4 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 23) ---
 □ 공단 콘텐츠 및 프로그램의 수정․보완 등 안정적 관리와 운영*
    * 운영에는 정보보안 및 개인정보보호 방안 및 법령준수 등이 포함됨...
--- Chunk 5 from 국민연금공단_2024년 이러닝시스템 운영 용역.hwp (Index: 72) ---
  ○ 기 개발된 공단 콘텐츠의 적기 업데이트로 지속적인 콘텐츠 품질 제고...

Generated Response:
콘텐츠 개발 및 관리 요구 사항은 다음과 같습니다:

1. **직무 콘텐츠 개발 및 운영**:
   - 직무 콘텐츠는 60차시로 구성되어야 하며, 부서별로 필수 콘텐츠 10차시가 추가로 개발되어야 합니다. 이러한 콘텐츠는 동영상 형태를 포함해야 합니다.

2. **안정적 관리와 운영**:
   - 공단의 콘텐츠 및 프로그램은 수정 및 보완이 필요하며, 이를 위한 안정적 관리가 요구됩니다. 이 운영에는 정보보안 및 개인정보 보호 방안이 포함되어야 하며, 관련 

In [33]:
# Answer the next query

query = "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘."

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp (Index: 38) ---
󰏅 시스템 운영 요구사항...
--- Chunk 2 from 국방과학연구소_대용량 자료전송시스템 고도화.hwp (Index: 31) ---
기술 및 기능 요구사항
▢ 시스템 및 시스템운영 요구사항...
--- Chunk 3 from 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.hwp (Index: 59) ---
나. 데이터 요구사항(Data Requirement)...
--- Chunk 4 from 수협중앙회_강릉어선안전조업국 상황관제시스템 구축.hwp (Index: 16) ---

□ 新 청사 상황실에 어선위치 모니터링 업무와 긴급상황 시 신속한 상황업무처리를 위하여 상황관제시스템 구축 필요
□ 실시간 해상 기상정보 및 어선 위치 등을 종합 모니터링하여 연근해 어선의 안전관리와 신속한 구조 지원을 위한 인프라 마련...
--- Chunk 5 from 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp (Index: 66) ---
 4. 현 예약발매시스템 업무 프로세스 및 정보기술 현황 분석
    * 업무 요구사항 분석서, 정보기술 요구사항 분석서 등 도출 ...

Generated Response:
제공된 문서에는 기초과학연구원 극저온시스템 사업의 AI 기반 예측에 대한 구체적인 요구사항이 포함되어 있지 않습니다. 그러나 "어선위치 모니터링 업무와 긴급상황 시 신속한 상황업무처리를 위한 상황관제시스템 구축 필요"와 관련하여 실시간 해상 기상정보 

In [34]:
# Answer the next query

query = "한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘."

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp (Index: 1) ---
1.1 사업 개요
가. 사업명 : 한국원자력연구원 선량평가시스템 고도화
나. 사업 기간 : 계약일로부터 6개월
다. 사업비 : 금 46,600 천원 (부가가치세 포함)
라. 입찰방법 : 제한경쟁입찰
○ 「소프트웨어산업진흥법 제24조의2 제2항」 및 「대기업인 소프트웨어 사업자가 참여할 수 있는 사업금액의 하한」에 따라 중소기업간 제한경쟁입찰
마. 계약방법 : 협상에 의한 계약 (기술평가90%, 가격평가10%)
○ 「국가를 당사자로 하는 계약에 관한 법률시행령 제43조(협상에 의한 계약체결) 및 제43조의2(지식기반사업의 계약방법)」에 의거 “협상에 의한 계약체결” 방법을 적용 
1.2. 추진배경 및 필요성
가. 규제요건 준수 필요
○ 원자력안전위원회고시 제2019-10호 ｢방사선방호 등에 관한 기준｣ 제16조(환경상의 위해방지)에서 제시하는 제한구역경계에서의 연간선량 준수 여부를 확인할 수 있는 체계 구축 필요
○ ICRP60 기반 평가장기 개선 필요
○ 액체유출물에 의한 주민피폭선...
--- Chunk 2 from 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp (Index: 8) ---
나. 사업 범위
 ㅇ 원내에서 관리하는 화학물질 운영현황의 통계 정보를 제공하는 시스템 구축
 ㅇ 고압가스 화학물질의 정보와 판매허가 업체를 관리하기 위한 시스템 구축
 ㅇ 고압가스 화학물질의 구매부터 안전성 검토 과정을 진행하기 위한 시스템 고도화
 ㅇ 연구실과 법적 준수 사항을 관리하기 위한 시스템 고도화
 ㅇ 화학제품 폐기·회수를 관리하기 위한 시스템 고도화...
--- Chunk 3 from 경상북도 봉화군_봉화군 재ᄂ

In [35]:
# Answer the next query

query = "고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래? 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘."

# Retrieve relevant chunks
retrieved_chunks = retrieve_chunks(query, k=5) # Retrieve top 5 chunks

if retrieved_chunks:
    print("Retrieved Chunks:")
    context_list = []
    for i, chunk in enumerate(retrieved_chunks):
        print(f"--- Chunk {i+1} from {chunk['filename']} (Index: {chunk['chunk_index']}) ---")
        print(chunk['text'][:500] + "...") # Print a snippet of the chunk
        context_list.append(chunk['text'])

    # Generate response
    response = generate_response(query, context_list)

    print("\nGenerated Response:")
    print(response)
else:
    print("No relevant chunks found to answer the query.")

Retrieved Chunks:
--- Chunk 1 from 서영대학교 산학협력단_전문대학 혁신지원사업 서영대학교 차세대 교육.hwp (Index: 54) ---
  라. 응용시스템 아키텍처
     업무변화에 신속 대응할 수 있는 웹 기반 서비스 지향 아키텍처 구현 및 최신 웹 기술 등을 검토하여 대학에 부합된 최적의 아키텍처를 적용 및 반영하여야 한다.
      - 교육혁신지원시스템 관련 시스템의 통합관리로 시스템 운영 및 유지보수 효율성 증대
      - 사용자의 이용 편의성과 접근성이 쉬운 아키텍처로 구성
      - 업무변화 및 관련 규정 변경을 유연하게 적용할 수 있도록 모듈화 구성
     업무담당자별 사용권한 관리가 가능토록 하여야 한다.
      - 시스템에 대한 권한은 통합으로 관리 되어야 하고, 역할(Role) 중심으로 배분하고, 역할별로 데이터에 대한 CRUD(Create, Read, Update, Delete) 권한을 부여할 수 있도록 구현
      - 부여된 권한대로 각 기능 범위 내에서 시스템 접근 및 사용
      - 사용자 및 권한별 화면 제공을 통해 부여되지 않은 기능에 접근하는 것을 원천적으로 ...
--- Chunk 2 from 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp (Index: 23) ---
 ◦ 향후 변화하는 발주기관의 특성을 고려하여 능동적이고 효율적인 시스템으로 구성하여야 한다.
 ◦ 시스템은 향후 확장성을 충분히 고려하여 설계·적용되어야 한다.
 ◦ 안정적, 효율적, 확장 가능한 최적 시스템으로 구축하며 사용자의 다양한 요구에 부흥하도록 구성하여야 한다.
 ◦ 수작업을 최소화하고, 가능한 전산으로 모든 업무를 처리할 수 있도록 시스템을 구성하여야 한다.
 ◦ 사용자의 편의성을 고려한 Work 